# GCcare NIST RAG API 테스트 노트북

이 노트북은 `FastAPI + LangServe`로 띄운 **/health**, **/rag** 엔드포인트를 간단히 테스트하기 위한 예시입니다.

## 0. 서버 실행 (터미널에서)

아나콘다 / venv를 활성화한 뒤, 프로젝트 루트(예: `.../gccare-langchain-rag-ai`)에서 아래 명령어를 **터미널에서** 실행해 FastAPI 서버를 먼저 띄워주세요.

```bash
python -m uvicorn api.serve_app:app --host 127.0.0.1 --port 8000 --reload
```

- 서버가 정상 실행되면: `http://127.0.0.1:8000/docs` 로도 접속 가능
- 이 노트북은 서버가 떠 있다는 가정하에 `http://127.0.0.1:8000` 으로 요청을 보냅니다.


In [1]:
import json
import textwrap
import requests

BASE_URL = "http://127.0.0.1:8000"

print("BASE_URL:", BASE_URL)

BASE_URL: http://127.0.0.1:8000


## 1. `/health` 엔드포인트 확인

FastAPI 서버가 제대로 떠 있는지 헬스 체크를 해봅니다.


In [2]:
health_res = requests.get(f"{BASE_URL}/health")
print("Status code:", health_res.status_code)
print("Response JSON:")
print(health_res.json())

Status code: 200
Response JSON:
{'status': 'ok'}


## 2. RAG `/rag/invoke` 기본 헬퍼 함수

LangServe가 마운트한 `/rag/invoke` 엔드포인트에 POST 요청을 보내는 헬퍼 함수를 만듭니다.

- body 형식: `{"input": "<질문 문자열>"}`


In [3]:
def ask_rag(question: str):
    """RAG 체인(/rag/invoke)에 질문을 던지고 결과를 반환하는 헬퍼 함수."""
    url = f"{BASE_URL}/rag/invoke"
    payload = {"input": question}
    print("\n=== 질문 =============================")
    print(question)
    print("=====================================")
    
    res = requests.post(url, json=payload)
    print("Status code:", res.status_code)
    try:
        data = res.json()
        print("\n원시 응답(JSON):")
        print(json.dumps(data, ensure_ascii=False, indent=2))
    except Exception:
        print("\n응답 본문(text):")
        print(res.text)
        
    return res

## 3. 기본 테스트 질문 1 — NIST AI RMF 목적

가장 기본적인 질문으로, NIST AI Risk Management Framework의 목적에 대해 물어봅니다.


In [4]:
_ = ask_rag("NIST AI Risk Management Framework의 핵심 목적이 뭐야?")


=== 질문 =============================
NIST AI Risk Management Framework의 핵심 목적이 뭐야?
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI Risk Management Framework의 핵심 목적은 인공지능 시스템의 위험을 관리하고 줄이는 방법을 제공하는 것입니다. 이를 통해 AI 시스템의 안전성과 신뢰성을 높이고, 윤리적이고 책임 있는 AI 사용을 촉진합니다.",
  "metadata": {
    "run_id": "86ccf1ef-acb1-4172-813f-c65d114468ec",
    "feedback_tokens": []
  }
}


## 4. 테스트 질문 2 — Governance 관련

거버넌스(거버넌스 기능, 역할 등)에 초점을 맞춰서 질문해 봅니다.


In [5]:
_ = ask_rag("NIST AI RMF에서 Governance(거버넌스)와 관련된 핵심 활동이나 역할을 정리해줘.")


=== 질문 =============================
NIST AI RMF에서 Governance(거버넌스)와 관련된 핵심 활동이나 역할을 정리해줘.
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI RMF에서 Governance(거버넌스)와 관련된 핵심 활동이나 역할은 다음과 같습니다:\n\n1. **정책 수립**: 조직의 AI 시스템에 대한 정책과 기준을 설정합니다.\n2. **위험 관리**: AI 시스템의 위험을 식별하고 평가하여 관리하는 프로세스를 수립합니다.\n3. **책임성 확보**: AI 시스템의 개발 및 운용에 대한 책임 있는 주체들을 명확히 하고 역할을 정의합니다.\n4. **모니터링 및 감사**: AI 시스템의 성능과 준수 상태를 지속적으로 모니터링하고 감사합니다.\n5. **이해관계자 참여**: 다양한 이해관계자와의 소통을 통해 의견을 수렴하고, 거버넌스 구조의 개선을 도모합니다.\n\n이러한 활동들은 AI 시스템의 신뢰성을 높이고, 윤리적 사용을 보장하는 데 기여합니다.",
  "metadata": {
    "run_id": "743f24f9-ddf9-44ef-82c2-0d9f186bdbc2",
    "feedback_tokens": []
  }
}


## 5. 테스트 질문 3 — 라이프사이클 관점 질문

NIST AI RMF의 구조(예: Map, Measure, Manage, Govern 등)에 초점.


In [6]:
_ = ask_rag("NIST AI RMF에서 Map, Measure, Manage, Govern 단계가 각각 무엇을 의미하는지 요약해줘.")


=== 질문 =============================
NIST AI RMF에서 Map, Measure, Manage, Govern 단계가 각각 무엇을 의미하는지 요약해줘.
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI RMF의 각 단계는 다음과 같습니다:\n\n1. **Map**: 인공지능 시스템의 맥락을 이해하고 관련된 이해관계자와 리스크를 파악하는 단계입니다.\n2. **Measure**: AI 시스템의 성능과 리스크를 평가하는 단계로, 적절한 메트릭스를 설정하고 측정합니다.\n3. **Manage**: 식별된 리스크를 관리하고 완화하기 위한 조치를 설계하고 실행하는 단계입니다.\n4. **Govern**: 전체 AI 시스템의 관리를 위한 정책과 프로세스를 수립하고, 지속적으로 모니터링하는 단계입니다.",
  "metadata": {
    "run_id": "124ba554-37b0-42ba-8d68-7bf39d00572a",
    "feedback_tokens": []
  }
}


## 6. 테스트 질문 4 — 실무 적용 관점 질문

실제로 조직이 AI 리스크 관리를 도입할 때의 적용 관점 질문입니다.


In [7]:
_ = ask_rag(
    "중견 교육기업이 NIST AI RMF를 참고해서 생성형 AI 서비스 리스크 관리를 하려면, 우선적으로 어떤 영역부터 점검하는 것이 좋을까?"
)


=== 질문 =============================
중견 교육기업이 NIST AI RMF를 참고해서 생성형 AI 서비스 리스크 관리를 하려면, 우선적으로 어떤 영역부터 점검하는 것이 좋을까?
Status code: 200

원시 응답(JSON):
{
  "output": "중견 교육기업이 NIST AI RMF를 참고하여 생성형 AI 서비스 리스크 관리를 시작할 때, 우선적으로 'AI 시스템 이해' 영역을 점검하는 것이 좋습니다. 이를 통해 AI 시스템의 목적, 작동 방식, 그리고 잠재적인 리스크 요인을 명확히 파악할 수 있습니다. 이후에는 데이터 관리, 성능 모니터링, 그리고 윤리적 고려 사항 등 다른 영역으로 점검을 이어나가야 합니다.",
  "metadata": {
    "run_id": "f8532a93-8444-4f0a-b17b-58b763f7058d",
    "feedback_tokens": []
  }
}


## 7. 테스트 질문 5 — 문서에 없을 법한 질문 (Out-of-Scope)

RAG가 **컨텍스트에 없는 내용은 모른다고 답하는지** 확인하기 위해 일부러 문서와 관계없는 질문을 던져봅니다.


In [8]:
_ = ask_rag("NIST AI RMF 문서에서 BTS 콘서트 일정에 대해 뭐라고 말하고 있어?")


=== 질문 =============================
NIST AI RMF 문서에서 BTS 콘서트 일정에 대해 뭐라고 말하고 있어?
Status code: 200

원시 응답(JSON):
{
  "output": "모릅니다. NIST AI RMF 문서에는 BTS 콘서트 일정에 대한 내용이 없습니다.",
  "metadata": {
    "run_id": "b2baf4ea-bf88-40ee-a3bd-97da80d6c74d",
    "feedback_tokens": []
  }
}


## 8. 자유 질문 셀

원하는 질문으로 직접 RAG 호출을 해볼 수 있는 셀입니다. `question` 변수만 바꿔서 실행하세요.


In [9]:
question = "AI 시스템의 신뢰성과 투명성을 동시에 높이기 위해 NIST AI RMF에서 제안하는 원칙들을 간단히 정리해줘."
_ = ask_rag(question)


=== 질문 =============================
AI 시스템의 신뢰성과 투명성을 동시에 높이기 위해 NIST AI RMF에서 제안하는 원칙들을 간단히 정리해줘.
Status code: 200

원시 응답(JSON):
{
  "output": "NIST AI RMF에서 제안하는 원칙들은 다음과 같습니다:\n\n1. **신뢰성**: AI 시스템의 정확성, 신뢰도 및 성능을 보장하여 사용자의 신뢰를 얻도록 함.\n2. **투명성**: AI 시스템의 작동 방식과 결정 과정을 이해할 수 있도록 정보 제공.\n3. **안전성**: 시스템이 안전하게 작동하도록 설계하고, 예상치 못한 상황에서도 올바르게 대응하도록 함.\n4. **지속적인 개선**: AI 시스템의 성능과 신뢰성을 지속적으로 평가하고 개선하여 변화하는 요구에 대응.\n5. **책임성**: AI 시스템의 결정과 행동에 대한 책임 소재를 명확히 하여 신뢰를 증진.\n\n이러한 원칙들을 통해 AI 시스템의 신뢰성과 투명성을 높이고자 합니다.",
  "metadata": {
    "run_id": "b4a4fb7d-7ee0-4fc4-99df-07cd0d6410cb",
    "feedback_tokens": []
  }
}


---
## 📌 추가 개인 실습 – EMR Copilot API 엔드포인트 설계 & 테스트

**목표**
- EMR 사용자(원장/간호사/접수)를 돕는 **미니 Copilot API 2~3개**를 설계한다.
- Insomnia/노트북에서 직접 요청을 보내 보며, “서비스 관점”에서 RAG/Agent를 바라본다.

**준비**
- 이전 시간에 만든 EMR RAG / Agent 체인 (또는 간단한 RAG 체인)
- 이 노트북에서 사용한 `requests` 기반 테스트 코드



---
### TODO 1. 엔드포인트 기획 – 어떤 Copilot을 줄 것인가?

예시 아이디어 (이 중 1~2개 골라 설계):

1. `POST /emr/faq`  
   - Request: `{ "role": "nurse" | "reception" | "doctor", "question": "..." }`  
   - Response: `{ "answer": "...", "source_docs": [...] }`

2. `POST /emr/summary`  
   - Request: `{ "free_text": "오늘 인수인계 메모..." }`  
   - Response: `{ "bullet_summary": ["...", "...", "..."] }`

3. `POST /emr/error-advice`  
   - Request: `{ "total_claims": 1200, "error_claims": 45 }`  
   - Response: `{ "error_rate": 3.75, "advice": "..." }`

> 각 엔드포인트별로 “누가, 어떤 화면/상황에서 쓸지” 간단히 메모해 두면 이후 기획 자료로 재사용하기 좋습니다.



---
### TODO 2. 노트북에서 API 호출 시나리오 작성하기

- 위에서 설계한 엔드포인트 중 최소 1개에 대해,
  - `requests.post()` 를 사용한 테스트 코드를 아래 빈 코드 셀에 작성해 보세요.
  - Request Body 예시를 2가지 이상 만들어, 응답 차이를 비교해 보세요.

- 응답에서 확인하고 싶은 포인트 예:
  - `role` 에 따라 답변 톤/내용이 달라지는지
  - `source_docs` 메타데이터(파일명/페이지)가 잘 노출되는지
  - 오류율 계산 등이 정확한지

> 이 노트북은 “진짜 프론트엔드가 붙기 전, 백엔드 Copilot API를 실험하는 샌드박스”라고 생각하고 활용해 보세요.


In [ ]:
# TODO: 설계한 EMR Copilot API를 호출하는 requests 예제를 작성해 보세요.
